In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

def simulate_rejections(test_type, n_samples, k, n_simulations=1000, alpha=0.05):
    rejections = 0
    for _ in range(n_simulations):
        data = np.random.chisquare(df=k, size=n_samples)
        if test_type == 'ks':
            d, p_value = stats.kstest(data, 'chi2', args=(k,))
        elif test_type == 'chi2':
            observed_freq, bins = np.histogram(data, bins='auto', density=True)
            expected_freq = len(data) * (stats.chi2.cdf(bins[1:], k) - stats.chi2.cdf(bins[:-1], k))
            chi2_stat, p_value = stats.chisquare(observed_freq * len(data), expected_freq)
        if p_value < alpha:
            rejections += 1
    return rejections / n_simulations

def simulate_rejections_with_pit(test_type, n_samples, k, n_simulations=1000, alpha=0.05):
    rejections = 0
    for _ in range(n_simulations):
        data = np.random.chisquare(df=k, size=n_samples)
        transformed_data = stats.chi2.cdf(data, df=k)
        if test_type == 'ks':
            d, p_value = stats.kstest(transformed_data, 'uniform')
        elif test_type == 'chi2':
            observed_freq, bins = np.histogram(transformed_data, bins='auto', density=True)
            expected_freq = len(data) * np.diff(bins)
            chi2_stat, p_value = stats.chisquare(observed_freq * len(data), expected_freq)
        if p_value < alpha:
            rejections += 1
    return rejections / n_simulations

SampleSize = [20, 30, 40, 50, 60, 70, 80, 90, 100]
df = [1, 75]
test_range = 1000
results = []

for size in SampleSize:
    for df_x in df:
        kologomorov_counter = 0
        kologomorov_pit_counter = 0
        chi2_counter = 0
        chi2_pit_counter = 0

        for i in range(test_range):
            data = np.random.chisquare(df_x, size)
            data_pit = stats.chi2.cdf(data, df=df_x)

            d, p_ks = stats.kstest(data, 'chi2', args=(df_x,))
            d_pit, p_ks_pit = stats.kstest(data_pit, 'uniform')

            bins = np.linspace(0, max(data), num=int(np.sqrt(size)))
            observed_freq, _ = np.histogram(data, bins=bins)
            expected_freq = len(data) * (stats.chi2.cdf(bins[1:], df_x) - stats.chi2.cdf(bins[:-1], df_x))
            chi2_stat, p_chi2 = stats.chisquare(observed_freq, expected_freq)

            bins_pit = np.linspace(0, 1, num=int(np.sqrt(size)))
            observed_freq_pit, _ = np.histogram(data_pit, bins=bins_pit)
            expected_freq_pit = np.full_like(observed_freq_pit, fill_value=len(data_pit)/len(bins_pit))

            chi2_pit_stat, p_chi2_pit = stats.chisquare(observed_freq_pit, expected_freq_pit)

            if p_ks <= 0.05:
                kologomorov_counter += 1
            if p_chi2 <= 0.05:
                chi2_counter += 1
            if p_ks_pit <= 0.05:
                kologomorov_pit_counter += 1
            if p_chi2_pit <= 0.05:
                chi2_pit_counter += 1

        results.append([kologomorov_counter/test_range, kologomorov_pit_counter/test_range, chi2_counter/test_range, chi2_pit_counter/test_range])

# Plot the results
results_array = np.array(results).reshape((len(SampleSize), len(df), 4))

for idx, df_x in enumerate(df):
    ks_results = results_array[:, idx, 0]
    ks_pit_results = results_array[:, idx, 1]
    chi2_results = results_array[:, idx, 2]
    chi2_pit_results = results_array[:, idx, 3]

    plt.figure(figsize=(8, 6))
    plt.plot(SampleSize, ks_results, label='K-S Test', marker='o')
    plt.plot(SampleSize, ks_pit_results, label='K-S Test with PIT', marker='o', linestyle='dashed')
    plt.plot(SampleSize, chi2_results, label='Chi-Square Test', marker='o')
    plt.plot(SampleSize, chi2_pit_results, label='Chi-Square Test with PIT', marker='o', linestyle='dashed')

    plt.title(f'Rejection Rates for df={df_x}')
    plt.xlabel('Sample Size')
    plt.ylabel('Rejection Rate')
    plt.ylim(0, 1)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()


ValueError: For each axis slice, the sum of the observed frequencies must agree with the sum of the expected frequencies to a relative tolerance of 1e-08, but the percent differences are:
0.008764176036662347